# docker-compose

## list

|コンテナ|サービス|port|
|---|---|---|
|MySQL|mysql|3306|
|MySQL|ssh|2222|
|openldap|ldap|389|
|openldap|ssh|2223|


## MySQL

### dockerhostの設定

In [36]:
%%bash
### mysqlクライアントのインストール(CentOS7)
sudo yum install -y http://dev.mysql.com/get/mysql-community-release-el7-5.noarch.rpm
sudo yum install -y mysql-community-client.x86_64

Loaded plugins: fastestmirror
Examining /var/tmp/yum-root-vps5IL/mysql-community-release-el7-5.noarch.rpm: mysql-community-release-el7-5.noarch
/var/tmp/yum-root-vps5IL/mysql-community-release-el7-5.noarch.rpm: does not update installed package.
Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: mirror.qoxy.com
 * epel: mirror.vinahost.vn
 * extras: mirror.nus.edu.sg
 * updates: mirror.qoxy.com
Package mysql-community-client-5.6.36-2.el7.x86_64 already installed and latest version
Nothing to do


Error: Nothing to do


### コンテナの作成

In [11]:
%%bash
### 変数の設定
container="mysql5.6_centos7"
docker_compose_file="docker-compose_${container}.yml"
mysql_port="3306"
ssh_port="2222"

### イメージ、コンテナが無い状態の確認
docker images
docker ps -a

### docker-composeディレクトリに移動
#work_dir="/home/jsd/work/build/docker-compose"
work_dir="${PYENV_DIR}/work/build/docker-compose"
cd ${work_dir}

### docker-composeファイルの作成
cat <<- EOS > ${docker_compose_file}
${container}:
   image: ftakao2007/mysql5.6_centos7:latest
   container_name: ${container}
   privileged: true
   tty: true
   ports:
     - "${mysql_port}:3306"
     - "${ssh_port}:22"
   #command: sh -c '/sbin/init'
   command: sh -c 'exec /usr/sbin/init && systemctl restart sshd.service && systemctl restart mysqld.service'
EOS

### コンテナの作成
docker-compose -f ${docker_compose_file} up -d
sleep 80

### イメージ、コンテナの確認
docker images
docker ps -a

### mysql接続
mysql -ujoruri -p'pass' -h 127.0.0.1 -P 3306 -e "SELECT CURDATE();" 2> /dev/null

REPOSITORY                    TAG                 IMAGE ID            CREATED             SIZE
ftakao2007/mail_centos7       latest              1e155bef8bad        2 hours ago         582MB
ftakao2007/openldap_centos7   latest              572b38993fa4        24 hours ago        596MB
ftakao2007/mysql5.6_centos7   latest              09a773ad8e34        2 days ago          2.16GB
ftakao2007/centos7            latest              0fede72fa1fe        2 days ago          488MB
CONTAINER ID        IMAGE                                COMMAND                  CREATED             STATUS              PORTS                                          NAMES
57fc8cd1f746        ftakao2007/openldap_centos7:latest   "sh -c 'exec /usr/..."   3 minutes ago       Up 3 minutes        0.0.0.0:389->389/tcp, 0.0.0.0:2223->22/tcp     openldap_centos7
ebd21afca604        ftakao2007/mysql5.6_centos7:latest   "sh -c 'exec /usr/..."   2 hours ago         Up 2 hours          0.0.0.0:3306->3306/tcp, 0.0.0.0:2222-

mysql5.6_centos7 is up-to-date


## LDAP

### dockerhostの設定

In [8]:
%%bash
### ldapクライアントのインストール(CentOS7)
sudo yum install -y openldap openldap-clients

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: mirror.nus.edu.sg
 * extras: download.nus.edu.sg
 * updates: mirror.qoxy.com
Package openldap-2.4.40-13.el7.x86_64 already installed and latest version
Resolving Dependencies
--> Running transaction check
---> Package openldap-clients.x86_64 0:2.4.40-13.el7 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                 Arch          Version                Repository   Size
Installing:
 openldap-clients        x86_64        2.4.40-13.el7          base        188 k

Transaction Summary
Install  1 Package

Total download size: 188 k
Installed size: 570 k
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : openldap-clients-2.4.40-13.el7.x86_64                        1/1 
  Verifying  : openldap-clients-2.4.40-13.el7.x86_64                        1/1 

Installed:
  openldap-clients.x86_64 0:2.4.40-13.el7        

In [12]:
%%bash
### 変数の設定
container="openldap_centos7"
docker_compose_file="docker-compose_${container}.yml"
ldap_port="389"
ssh_port="2223"
dn="dc=dockernet,dc=co,dc=jp"
search="(&(objectClass=person)(mail=joruri_test11@dockernet.co.jp))"

### イメージ、コンテナが無い状態の確認
docker images
docker ps -a

### docker-composeディレクトリに移動
#work_dir="/home/jsd/work/build/docker-compose"
work_dir="${PYENV_DIR}/work/build/docker-compose"
cd ${work_dir}

### docker-composeファイルの作成
cat <<- EOS > ${docker_compose_file}
${container}:
   image: ftakao2007/openldap_centos7:latest
   container_name: ${container}
   privileged: true
   tty: true
   ports:
     - "${ldap_port}:389"
     - "${ssh_port}:22"
   #command: sh -c '/sbin/init'
   command: sh -c 'exec /usr/sbin/init && systemctl restart sshd.service && systemctl restart slapd.service'
EOS

### コンテナの作成
docker-compose -f ${docker_compose_file} up -d
#sleep 80

### イメージ、コンテナの確認
docker images
docker ps -a

### ldap接続確認
ldapsearch -x -LLL -b "${dn}" -h 127.0.0.1 "${search}"

REPOSITORY                    TAG                 IMAGE ID            CREATED             SIZE
ftakao2007/mail_centos7       latest              1e155bef8bad        2 hours ago         582MB
ftakao2007/openldap_centos7   latest              572b38993fa4        24 hours ago        596MB
ftakao2007/mysql5.6_centos7   latest              09a773ad8e34        2 days ago          2.16GB
ftakao2007/centos7            latest              0fede72fa1fe        2 days ago          488MB
CONTAINER ID        IMAGE                                COMMAND                  CREATED             STATUS              PORTS                                          NAMES
57fc8cd1f746        ftakao2007/openldap_centos7:latest   "sh -c 'exec /usr/..."   5 minutes ago       Up 5 minutes        0.0.0.0:389->389/tcp, 0.0.0.0:2223->22/tcp     openldap_centos7
ebd21afca604        ftakao2007/mysql5.6_centos7:latest   "sh -c 'exec /usr/..."   2 hours ago         Up 2 hours          0.0.0.0:3306->3306/tcp, 0.0.0.0:2222-

openldap_centos7 is up-to-date


## Mailサーバ

### imap,smtp,ldap連携

In [39]:
%%bash
### 変数の設定
container="mail_server_centos7"
docker_compose_file="docker-compose_${container}.yml"
smtp_port="2525"
imap_port="1143"
ssh_port="2224"

### イメージ、コンテナが無い状態の確認
docker images
docker ps -a

### docker-composeディレクトリに移動
#work_dir="/home/jsd/work/build/docker-compose"
work_dir="${PYENV_DIR}/work/build/docker-compose"
cd ${work_dir}

### docker-composeファイルの作成
cat <<- EOS > ${docker_compose_file}
${container}:
   image: ftakao2007/mail_centos7:latest
   container_name: ${container}
   privileged: true
   tty: true
   ports:
     - "${smtp_port}:25"
     - "${imap_port}:143"
     - "${ssh_port}:22"
   #command: sh -c '/sbin/init'
   command: sh -c 'exec /usr/sbin/init && systemctl restart sshd.service && systemctl restart postfix.service && systemctl restart dovecot.service'
EOS

### コンテナの作成
docker-compose -f ${docker_compose_file} up -d
#sleep 80

### イメージ、コンテナの確認
docker images
docker ps -a

REPOSITORY                    TAG                 IMAGE ID            CREATED             SIZE
ftakao2007/mail_centos7       latest              1e155bef8bad        14 hours ago        582MB
ftakao2007/openldap_centos7   latest              572b38993fa4        35 hours ago        596MB
ftakao2007/mysql5.6_centos7   latest              09a773ad8e34        3 days ago          2.16GB
ftakao2007/centos7            latest              0fede72fa1fe        3 days ago          488MB
CONTAINER ID        IMAGE                                COMMAND                  CREATED             STATUS              PORTS                                          NAMES
57fc8cd1f746        ftakao2007/openldap_centos7:latest   "sh -c 'exec /usr/..."   12 hours ago        Up 12 hours         0.0.0.0:389->389/tcp, 0.0.0.0:2223->22/tcp     openldap_centos7
ebd21afca604        ftakao2007/mysql5.6_centos7:latest   "sh -c 'exec /usr/..."   14 hours ago        Up 14 hours         0.0.0.0:3306->3306/tcp, 0.0.0.0:2222-

Creating mail_server_centos7


In [28]:
%%bash
### mailxのインストール
sudo yum install mailx fetchmail procmail

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: mirror.qoxy.com
 * epel: ftp.cuhk.edu.hk
 * extras: mirror.nus.edu.sg
 * updates: mirror.qoxy.com
Package mailx-12.5-12.el7_0.x86_64 already installed and latest version
Package fetchmail-6.3.24-5.el7.x86_64 already installed and latest version
Package procmail-3.22-35.el7.x86_64 already installed and latest version
Nothing to do


In [32]:
%%bash
### 設定
mail_server="192.168.33.11"
smtp_port="2525"
imap_port="1143"
mail_from="joruri_test12@dockernet.co.jp"
mail_to="joruri_test11@dockernet.co.jp"

### メール送信テスト
echo "mail body" | mail -s "mail title" -S "smtp=smtp://${mail_server}:${smtp_port}" -r ${mail_from} ${mail_to}

In [34]:
%%bash
### 設定
mail_server="192.168.33.11"
imap_port="1143"

### procmailの設定
cat <<- EOS > ~/.fetchmailrc
poll ${mail_server}
  protocol imap
  port ${imap_port}
  username "0050011"
  password "pass"
  mda "/usr/local/bin/procmail"
EOS

### postfixの設定変更。メールサーバを既存で使ってる場合は設定に注意
sudo sed -i -e 's/#mailbox_command = \/some\/where\/procmail$/mailbox_command = \/usr\/bin\/procmail/' /etc/postfix/main.cf

### メールが届いていることの確認
fetchmail -c

3 messages (1 seen) for 0050011 at 192.168.33.11.


### imap,smtp,ldap連携 + Joruriのvirtualプラグイン設定

In [40]:
%%bash
### 変数の設定
container="mail_server_virtual_for_jorurimail_centos7"
docker_compose_file="docker-compose_${container}.yml"
smtp_port="2526"
imap_port="1144"
ssh_port="2225"

### イメージ、コンテナが無い状態の確認
docker images
docker ps -a

### docker-composeディレクトリに移動
#work_dir="/home/jsd/work/build/docker-compose"
work_dir="${PYENV_DIR}/work/build/docker-compose"
cd ${work_dir}

### docker-composeファイルの作成
cat <<- EOS > ${docker_compose_file}
${container}:
   image: ftakao2007/mail_centos7:virtual_for_jorurimail
   container_name: ${container}
   privileged: true
   tty: true
   ports:
     - "${smtp_port}:25"
     - "${imap_port}:143"
     - "${ssh_port}:22"
   #command: sh -c '/sbin/init'
   command: sh -c 'exec /usr/sbin/init && systemctl restart sshd.service && systemctl restart postfix.service && systemctl restart dovecot.service'
EOS

### コンテナの作成
docker-compose -f ${docker_compose_file} up -d
#sleep 80

### イメージ、コンテナの確認
docker images
docker ps -a

REPOSITORY                    TAG                 IMAGE ID            CREATED             SIZE
ftakao2007/openldap_centos7   latest              572b38993fa4        39 hours ago        596MB
ftakao2007/mysql5.6_centos7   latest              09a773ad8e34        3 days ago          2.16GB
ftakao2007/centos7            latest              0fede72fa1fe        3 days ago          488MB
CONTAINER ID        IMAGE                                COMMAND                  CREATED             STATUS              PORTS                                          NAMES
57fc8cd1f746        ftakao2007/openldap_centos7:latest   "sh -c 'exec /usr/..."   15 hours ago        Up 15 hours         0.0.0.0:389->389/tcp, 0.0.0.0:2223->22/tcp     openldap_centos7
ebd21afca604        ftakao2007/mysql5.6_centos7:latest   "sh -c 'exec /usr/..."   17 hours ago        Up 17 hours         0.0.0.0:3306->3306/tcp, 0.0.0.0:2222->22/tcp   mysql5.6_centos7
virtual_for_jorurimail: Pulling from ftakao2007/mail_centos7
Digest: 

Pulling mail_server_virtual_for_jorurimail_centos7 (ftakao2007/mail_centos7:virtual_for_jorurimail)...
Creating mail_server_virtual_for_jorurimail_centos7
